In [1]:
import json 

In [2]:
path_tasks = "/root/seele-slime/agent/valid_tasks.json"


In [3]:
def generate_sample(task_name, i):
    schema = {
        "prompt": [{"role": "user", "content": ""}],
        "label": "None",
        "metadata": {
            "split": "train",
            "index": i,
            "answer": "None",
            "env_name": task_name
        }
    }
    
    return schema

In [4]:
with open(path_tasks, mode="r") as f:
    data = json.load(f)
    f.close()
    
print(len(data))
all_samples = [generate_sample(name, idx) for idx, name in enumerate(data)]
    

124


In [5]:
print(all_samples[1])

{'prompt': [{'role': 'user', 'content': ''}], 'label': 'None', 'metadata': {'split': 'train', 'index': 1, 'answer': 'None', 'env_name': 'test-financial-document-processor'}}


In [ ]:
save_name = "agent_gym_data.jsonl"
with open(save_name, 'w', encoding='utf-8') as f:
    for item in all_samples:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

: 

In [2]:
from transformers import AutoTokenizer


qwen_tokenizer_path = "/root/qwen3-coder-30b-agent/qwen3-coder-30b"
tokenizer = AutoTokenizer.from_pretrained(qwen_tokenizer_path)

print(tokenizer.eos_token)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


<|im_end|>


In [3]:
print([tokenizer.apply_chat_template([{"role": "user", "content": "What is the meaning of life?"}], tokenize=False, add_generation_prompt=True)])

['<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\n']


In [5]:
test_data = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the meaning of life?"},
    {"role": "assistant", "content": "The meaning of life is to live."},
    {"role": "user", "content": "Calculate the sum of 1 + 1."},
    {"role": "assistant", "content": "The answer is 2."}
]

prompt1 = test_data[:2]

print([tokenizer.apply_chat_template(prompt1, tokenize=False, add_generation_prompt=True)])

data1 = test_data[:3]

print([tokenizer.apply_chat_template(data1, tokenize=False, add_generation_prompt=False)])

prompt2 = test_data[:4]

print([tokenizer.apply_chat_template(prompt2, tokenize=False, add_generation_prompt=True)])

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\n']
['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\nThe meaning of life is to live.<|im_end|>\n']
['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\nThe meaning of life is to live.<|im_end|>\n<|im_start|>user\nCalculate the sum of 1 + 1.<|im_end|>\n<|im_start|>assistant\n']


In [13]:
print(tokenizer.apply_chat_template([{"role": "user", "content": "hello"}, {"role": "assistant", "content": "hi"}], add_generation_prompt=False))
print(tokenizer.apply_chat_template([{"role": "assistant", "content": "hi"}], add_generation_prompt=False))


[151644, 872, 198, 14990, 151645, 198, 151644, 77091, 198, 6023, 151645, 198]
[151644, 77091, 198, 6023, 151645, 198]


In [ ]:
print([tokenizer.decode([198])])
print(tokenizer.encode("\n"))

['\n']
[198]


: 

In [21]:
assistant_token = tokenizer.encode(test_data[2]["content"] + "<|im_end|>")
sep = tokenizer.encode("\n")
prompt_token = tokenizer.apply_chat_template(prompt1, tokenize=True, add_generation_prompt=True)

print([tokenizer.decode(prompt_token+assistant_token+sep)])

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\nThe meaning of life is to live.<|im_end|>\n']


In [25]:
prompt_token2 = tokenizer.apply_chat_template([test_data[3]], add_generation_prompt=True, tokenize=True)
print([tokenizer.decode(prompt_token+assistant_token+sep+prompt_token2)])

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the meaning of life?<|im_end|>\n<|im_start|>assistant\nThe meaning of life is to live.<|im_end|>\n<|im_start|>user\nCalculate the sum of 1 + 1.<|im_end|>\n<|im_start|>assistant\n']


In [37]:
# Maximum overlap sub-sequence

token_seq1 = [151644, 872, 198, 14990, 151645, 198, 151644, 77091, 198, 6023, 151645, 198]
token_seq2 = [151644, 77091, 198, 6023, 151645, 198]

ptr1 = 0
ptr2 = 0

for index in range(len(token_seq1)):
    ptr1 = index
    ptr2 = 0
    while token_seq1[ptr1] == token_seq2[ptr2]:
        ptr1 += 1
        ptr2 += 1
        if not (ptr1 < len(token_seq1) - 1 and ptr2 < len(token_seq2) - 1):
            break
        
    if ptr2 == len(token_seq2)-1:
        print(index)
        break

6
